In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("db_python.sqlite")
cursor = conn.cursor()

Criando as tabelas e inserindo os dados no banco de dados "db_python.sqlite"

In [3]:
with open('python_sql_tables.sql', 'r', encoding='utf-8') as file:
    cursor.executescript(file.read())


OperationalError: table Clientes already exists

## -- EXERCÍCIOS --



1. Fácil: Selecione todos os clientes cadastrados após 01/01/2023.

In [ ]:
cursor.execute(
    '''SELECT * FROM Clientes
    WHERE DataCadastro > '2023-01-01'
''')
cursor.fetchall()

[(2, 'Maria Santos', 'maria@email.com', '2023-01-10'),
 (3, 'Pedro Oliveira', 'pedro@email.com', '2023-03-22'),
 (5, 'Carlos Ferreira', 'carlos@email.com', '2023-02-14'),
 (7, 'Fernando Costa', 'fernando@email.com', '2023-04-05'),
 (9, 'Ricardo Nunes', 'ricardo@email.com', '2023-05-01')]

2. Fácil: Liste os 5 produtos mais caros.

In [ ]:
cursor.execute(
    '''SELECT *
    FROM Produtos
    ORDER BY Preco DESC LIMIT 5;
''')
cursor.fetchall()

[(2, 'Notebook ABC', 1499.99, 'Eletrônicos'),
 (1, 'Smartphone XYZ', 999.99, 'Eletrônicos'),
 (10, 'Cadeira de Escritório', 249.99, 'Móveis'),
 (9, 'Relógio Inteligente', 199.99, 'Eletrônicos'),
 (3, 'Tênis Runner', 129.99, 'Calçados')]

3. Médio: Calcule o total de vendas (quantidade * preço) para cada produto.

In [ ]:
cursor.execute(
    '''SELECT Produtos.ProdutoID, Produtos.Nome, (ItensPedido.Quantidade * ItensPedido.PrecoUnitario) AS total_vendas
    FROM Produtos
    JOIN ItensPedido ON Produtos.ProdutoID = ItensPedido.ProdutoID
    GROUP BY ItensPedido.ProdutoID
''')
cursor.fetchall()

[(1, 'Smartphone XYZ', 999.99),
 (2, 'Notebook ABC', 1499.99),
 (3, 'Tênis Runner', 129.99),
 (4, 'Camiseta Básica', 89.97),
 (5, 'Fones de Ouvido', 179.98),
 (6, 'Livro "SQL Avançado"', 59.99),
 (7, 'Cafeteira Elétrica', 79.99),
 (8, 'Mochila Escolar', 99.98),
 (9, 'Relógio Inteligente', 199.99),
 (10, 'Cadeira de Escritório', 249.99)]

4. Médio: Encontre os clientes que fizeram pelo menos 2 pedidos.

In [ ]:
cursor.execute(
    '''SELECT Clientes.ClienteID, Clientes.Nome, COUNT(Pedidos.ClienteID) AS qtd_pedidos
    FROM Clientes
    JOIN Pedidos ON Clientes.ClienteID = Pedidos.ClienteID
    GROUP BY Pedidos.ClienteID
    HAVING qtd_pedidos >= 2;
''')
for cliente in cursor.fetchall():
    print(f"ClienteID: {cliente[0]}, Cliente: {cliente[1]}, Quantidade de pedidos: {cliente[2]}")

ClienteID: 1, Cliente: João Silva, Quantidade de pedidos: 2
ClienteID: 2, Cliente: Maria Santos, Quantidade de pedidos: 2
ClienteID: 3, Cliente: Pedro Oliveira, Quantidade de pedidos: 2
ClienteID: 4, Cliente: Ana Rodrigues, Quantidade de pedidos: 2
ClienteID: 5, Cliente: Carlos Ferreira, Quantidade de pedidos: 2


5. Médio: Liste as categorias de produtos e a quantidade de produtos em cada categoria.

In [ ]:
cursor.execute(
    '''SELECT Produtos.Categoria, COUNT(Categoria) as qtd_produtos
    FROM Produtos
    GROUP BY Categoria;
''')
cursor.fetchall()

[('Acessórios', 1),
 ('Calçados', 1),
 ('Eletrodomésticos', 1),
 ('Eletrônicos', 4),
 ('Livros', 1),
 ('Móveis', 1),
 ('Roupas', 1)]

6. Difícil: Encontre o valor total gasto por cada cliente, ordenado do maior para o menor.
-- Dica: Você precisará juntar as tabelas Pedidos, ItensPedido e Clientes.

In [ ]:
cursor.execute(
    '''SELECT Clientes.ClienteID, Clientes.Nome, ROUND(SUM(ItensPedido.Quantidade * ItensPedido.PrecoUnitario), 2) AS valor_total
    FROM Clientes
    JOIN Pedidos ON Clientes.ClienteID = Pedidos.ClienteID
    JOIN ItensPedido ON Pedidos.PedidoID = ItensPedido.PedidoID
    GROUP BY Clientes.ClienteID
    ORDER BY valor_total DESC;
''')
cursor.fetchall()

[(3, 'Pedro Oliveira', 2689.96),
 (1, 'João Silva', 2179.96),
 (4, 'Ana Rodrigues', 1859.94),
 (7, 'Fernando Costa', 1499.99),
 (5, 'Carlos Ferreira', 649.95),
 (2, 'Maria Santos', 479.94),
 (10, 'Beatriz Lima', 399.96),
 (9, 'Ricardo Nunes', 279.98),
 (6, 'Lucia Pereira', 149.98),
 (8, 'Mariana Alves', 149.95)]

7. Difícil: Identifique os produtos que nunca foram pedidos.
-- Dica: Utilize uma subconsulta ou LEFT JOIN.


In [ ]:
cursor.execute(
    '''SELECT Produtos.ProdutoID, Produtos.Nome
    FROM Produtos
    LEFT JOIN ItensPedido ON Produtos.ProdutoID = ItensPedido.ProdutoID
    GROUP BY ItensPedido.ProdutoID
    HAVING COUNT(ItensPedido.PedidoID) == 0;
''')
cursor.fetchall()

[]

8. Difícil: Calcule a média de itens por pedido para cada cliente.

In [ ]:
cursor.execute(
    '''SELECT Clientes.ClienteID, Clientes.Nome, (SUM(ItensPedido.Quantidade)/COUNT(ItensPedido.PedidoID)) AS itens_por_pedido
    FROM Clientes
    JOIN Pedidos ON Clientes.ClienteID = Pedidos.ClienteID
    JOIN ItensPedido ON Pedidos.PedidoID = ItensPedido.PedidoID
    GROUP BY Clientes.ClienteID;
''')
cursor.fetchall()

[(1, 'João Silva', 1),
 (2, 'Maria Santos', 2),
 (3, 'Pedro Oliveira', 1),
 (4, 'Ana Rodrigues', 1),
 (5, 'Carlos Ferreira', 1),
 (6, 'Lucia Pereira', 1),
 (7, 'Fernando Costa', 1),
 (8, 'Mariana Alves', 5),
 (9, 'Ricardo Nunes', 1),
 (10, 'Beatriz Lima', 2)]

9. Muito Difícil: Liste os top 3 clientes que gastaram mais em cada mês de 2023.
-- Dica: Você precisará usar window functions como ROW_NUMBER() ou RANK().


In [36]:
cursor.execute(
    '''WITH VendasMensais AS (
    SELECT Clientes.ClienteID,
    Clientes.Nome,
    STRFTIME('%m', Pedidos.DataPedido) AS Mes,
    STRFTIME('%Y', Pedidos.DataPedido) AS Ano,
    SUM(ItensPedido.Quantidade * ItensPedido.PrecoUnitario) as Total_Gasto
    FROM Clientes
    JOIN Pedidos ON Clientes.ClienteID = Pedidos.ClienteID
    JOIN ItensPedido ON Pedidos.PedidoID = ItensPedido.PedidoID
    WHERE Ano = '2023'
    GROUP BY Clientes.ClienteID, Ano, Mes
    ),
    RankingMensal AS (
    SELECT ClienteID,
    Nome,
    Mes,
    Ano,
    total_gasto,
    ROW_NUMBER() OVER (PARTITION BY Ano, Mes ORDER BY total_gasto DESC) AS Ranking
    FROM VendasMensais
    )
    SELECT Ano,
        Mes,
        Nome,
        Ranking,
        Total_Gasto
    FROM RankingMensal
    WHERE Ranking <= 3
''')
cursor.fetchall()

[('2023', '01', 'João Silva', 1, 1179.97),
 ('2023', '02', 'Maria Santos', 1, 219.96),
 ('2023', '03', 'Pedro Oliveira', 1, 1559.98),
 ('2023', '04', 'Carlos Ferreira', 1, 449.98),
 ('2023', '04', 'Ana Rodrigues', 2, 179.97),
 ('2023', '05', 'João Silva', 1, 999.99),
 ('2023', '05', 'Maria Santos', 2, 259.98),
 ('2023', '06', 'Fernando Costa', 1, 1499.99),
 ('2023', '06', 'Beatriz Lima', 2, 399.96000000000004),
 ('2023', '06', 'Ricardo Nunes', 3, 279.98),
 ('2023', '07', 'Ana Rodrigues', 1, 1679.97),
 ('2023', '07', 'Pedro Oliveira', 2, 1129.98),
 ('2023', '07', 'Carlos Ferreira', 3, 199.97)]

10. Muito Difícil: Para cada produto, calcule a porcentagem de vendas que ele representa em relação ao total de vendas da sua categoria.
-- Dica: Utilize CTEs (Common Table Expressions) e window functions.

In [48]:
cursor.execute(
    '''WITH VendasTotalCategoria AS (
    SELECT Produtos.Categoria,
    Produtos.ProdutoID,
    SUM(ItensPedido.Quantidade * ItensPedido.PrecoUnitario) AS VendasCategoria
    FROM ItensPedido
    JOIN Produtos ON ItensPedido.ProdutoID = Produtos.ProdutoID
    GROUP BY Produtos.Categoria
    ),
    TotalProdutos AS (
    SELECT Produtos.ProdutoID,
    Produtos.Nome AS Nome,
    SUM(ItensPedido.Quantidade * ItensPedido.PrecoUnitario) AS VendasProduto,
    Produtos.Categoria
    FROM Produtos
    JOIN ItensPedido ON Produtos.ProdutoID = ItensPedido.ProdutoID
    GROUP BY ItensPedido.ProdutoID
    )
    SELECT 
    TotalProdutos.ProdutoID,
    TotalProdutos.Nome,
    VendasTotalCategoria.Categoria,
    ((TotalProdutos.VendasProduto / VendasCategoria) * 100) AS PorcentagemProdutoCategoria
    FROM TotalProdutos
    JOIN Produtos ON TotalProdutos.ProdutoID = Produtos.ProdutoID
    JOIN VendasTotalCategoria ON Produtos.Categoria = VendasTotalCategoria.Categoria
    GROUP BY TotalProdutos.ProdutoID, VendasTotalCategoria.Categoria
    '''
)
cursor.fetchall()

[(1, 'Smartphone XYZ', 'Eletrônicos', 35.92834379457405),
 (2, 'Notebook ABC', 'Eletrônicos', 53.89269533537647),
 (3, 'Tênis Runner', 'Calçados', 100.0),
 (4, 'Camiseta Básica', 'Roupas', 100.0),
 (5, 'Fones de Ouvido', 'Eletrônicos', 5.388706650522702),
 (6, 'Livro "SQL Avançado"', 'Livros', 100.0),
 (7, 'Cafeteira Elétrica', 'Eletrodomésticos', 100.0),
 (8, 'Mochila Escolar', 'Acessórios', 100.0),
 (9, 'Relógio Inteligente', 'Eletrônicos', 4.790254219526771),
 (10, 'Cadeira de Escritório', 'Móveis', 100.0)]

In [42]:
cursor.execute(
    '''
    SELECT Produtos.Categoria,
    Produtos.ProdutoID,
    SUM(ItensPedido.Quantidade * ItensPedido.PrecoUnitario) AS VendasCategoria
    FROM ItensPedido
    JOIN Produtos ON ItensPedido.ProdutoID = Produtos.ProdutoID
    GROUP BY Produtos.Categoria
    '''
)
cursor.fetchall()

[('Acessórios', 8, 249.95),
 ('Calçados', 3, 519.96),
 ('Eletrodomésticos', 7, 239.96999999999997),
 ('Eletrônicos', 1, 8349.87),
 ('Livros', 6, 239.96),
 ('Móveis', 10, 499.98),
 ('Roupas', 4, 239.92)]

Fechando conexão

In [ ]:
conn.close()